In [3]:
# Mine vs Rock

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Reading the dataset
# Get the dataset "sonar.csv" at 
# https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data
# Just save the file as sonar.csv
def read_dataset(file_name):
    dir_path = ""
    df = pd.read_csv(dir_path+file_name,delimiter=',')
    print("Nbr columns: ",len(df.columns))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
    # Encode the dependant variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    Y = one_hot_encode(y)
    print("X.shape",X.shape)
    print("Y.shape",Y.shape)
    return (X,Y)
 
# Define the encoder function M => 1, R => 0
def one_hot_encode(labels):
    n_labels = len(labels)
#     n_unique_labels = len(np.unique(labels))
    n_unique_labels = 2
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode
 
# Read the dataset
X, Y = read_dataset("sonar.csv")

# Shuffle the dataset to mix up the rows
X, Y = shuffle(X, Y, random_state=1)
 
# Convert the dataset into train and test datasets
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=415)

print()
print("-"*25)
for i in range(93,101):
    print(i,"Original Class:",int(Y[i][1]))
print("-"*25)
print()

# Inspect the shape of the train and test datasets
print("train_x.shape",train_x.shape)
print("train_y.shape",train_y.shape)
print("test_x.shape",test_x.shape)
print("test_y.shape",test_y.shape)
 
# Define the hyperparameters
learning_rate = 0.3
training_epochs = 1000
cost_history = np.empty(shape=[1], dtype=float)
# Number of features <=> number of columns
n_dim = X.shape[1] 
print("n_dim",n_dim)
print()

n_class = 2
model_path = "/tensorflow_basic_tutorial/model/"
 
# Define the number of hidden layers an the
# number of neurons for each layer
n_hidden_1 = 60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4 = 60
 
# Inputs and outputs
x = tf.placeholder(tf.float32,[None, n_dim])
y_ = tf.placeholder(tf.float32,[None, n_class])
  
# Model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activations
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with sigmoid activations
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoid activations
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with RELU activations
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    # Output layer with linear activations
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out_b']
    return out_layer
 
# define the weights and the biases for each layer
 
weights = {
    'h1': tf.Variable(tf.truncated_normal([n_dim, n_hidden_1]),name="h1"),
    'h2': tf.Variable(tf.truncated_normal([n_hidden_1, n_hidden_2]),name="h2"),
    'h3': tf.Variable(tf.truncated_normal([n_hidden_2, n_hidden_3]),name="h3"),
    'h4': tf.Variable(tf.truncated_normal([n_hidden_3, n_hidden_4]),name="h4"),
    'out': tf.Variable(tf.truncated_normal([n_hidden_4, n_class]),name="out"),
    }
biases = {
    'b1': tf.Variable(tf.truncated_normal([n_hidden_1]),name="b1"),
    'b2': tf.Variable(tf.truncated_normal([n_hidden_2]),name="b2"),
    'b3': tf.Variable(tf.truncated_normal([n_hidden_3]),name="b3"),
    'b4': tf.Variable(tf.truncated_normal([n_hidden_4]),name="b4"),
    'out_b': tf.Variable(tf.truncated_normal([n_class]),name="out_b"),
    }
 
# Initialization
init = tf.global_variables_initializer()
 
saver = tf.train.Saver()
 
# Call your model defined
y = multilayer_perceptron(x, weights, biases)
 
# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# Launch the graph
sess = tf.Session()
sess.run(init)

mse_history = []
accuracy_history = []

# Calculate the cost and the accuracy for each epoch
for epoch in range(training_epochs):
    sess.run(training_step, feed_dict={x:train_x, y_:train_y})
    cost = sess.run(cost_function,feed_dict={x:train_x, y_:train_y})
    cost_history = np.append(cost_history, cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#     print("Accuracy: ", (sess.run(accuracy, feed_dict={x:test_x, y_:test_y})))
    pred_y = sess.run(y,feed_dict={x:test_x} )
    mse = tf.reduce_mean(tf.square(pred_y - test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy,feed_dict={x:train_x, y_:train_y}))
    accuracy_history.append(accuracy)
    if epoch % 50 == 0:
        print()
        print('epoch: ', epoch,' - ', 'cost: ', cost, " - MSE: ", mse_, "- Train Accuracy: ", accuracy)
    else:
        print( '.', end='' )
    print()
    
save_path = saver.save(sess,model_path+"model.ckpt")
print("Model saved in file: %s", save_path)

print()
print("-"*90)
print()

# Plot training accuracy graph
plt.figure(figsize=(10,7))
plt.plot(accuracy_history)
plt.xlabel("Epochs")
plt.ylabel("Training Accuracy")
plt.show()

print()
print("-"*90)
print()

# Filter the mse_history to remove the outliers
filtered_mse_history = []
for mse in mse_history:
    # remove the outliers where mse >= 50
    if mse < 50:
        filtered_mse_history.append(mse) 

# Plot filtered MSE graph
plt.figure(figsize=(10,7))
plt.plot(filtered_mse_history)
plt.xlabel("Epochs")
plt.ylabel("MSE - mean square error")
plt.show()
print()
print("-"*90)
print()

# Print the final accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Test Accuracy: ", (sess.run(accuracy, feed_dict={x:test_x, y_:test_y} )))

# Print the final mean square error (MSE)
pred_y = sess.run(y, feed_dict={x:test_x})
mse = tf.reduce_mean(tf.square(pred_y- test_y))
print("MSE: %.4f" % sess.run(mse))

#--------------

Nbr columns:  61
X.shape (207, 60)
Y.shape (207, 2)

-------------------------
93 Original Class: 1
94 Original Class: 0
95 Original Class: 0
96 Original Class: 1
97 Original Class: 0
98 Original Class: 1
99 Original Class: 0
100 Original Class: 0
-------------------------

train_x.shape (165, 60)
train_y.shape (165, 2)
test_x.shape (42, 60)
test_y.shape (42, 2)
n_dim 60

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


epoch:  0  -  cost:  113.04522  - MSE:  12014.264757060117 - Train Accuracy:  0.45454547

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.

epoch:  50  -  cost:  0.6564528  - MSE:  1.3274317488727108 - Train Accuracy:  0.6424242

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.

epoch:  100  -  cost:  0.4041234  - MSE:  1.05046566640

ValueError: Parent directory of /Users/claudecoulombe/git/tensorflow_basic_tutorial/model/model.ckpt doesn't exist, can't save.

In [99]:
print("***************** Prediction on testing data *****************")
print("**************************************************************")
print("* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *")
print("**************************************************************")
accuracy = sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y,1),tf.argmax(y_,1)), tf.float32)), feed_dict={x:test_x, y_:test_y})
print('Test accuracy',str(round(float(accuracy),3)*100)+"%")
print("Other mean to evaluate the test accuracy...")
predicted_targets = sess.run(tf.argmax(y,1), feed_dict={x:test_x}).tolist()
true_targets = [np.argmax(a_data_sample) for a_data_sample in test_y]
for true_target,predicted_target in zip(true_targets,predicted_targets):
    print("True target:",true_target, "Predicted target:",predicted_target,"Accurate prediction:",true_target==predicted_target)
total_accurate = sum([1 for index in range(len(true_targets)) if true_targets[index] == predicted_targets[index]])
print("Test accuracy:",str(round(total_accurate/len(true_targets),3)*100)+"%")

***************** Prediction on testing data *****************
**************************************************************
* 1 stands for M, (i.e. Mine) and 0 stands for R (i.e. Rock) *
**************************************************************
Test accuracy 85.7%
Other mean to evaluate the test accuracy...
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 0 Predicted target: 0 Accurate prediction: True
True target: 1 Predicted target: 1 Accurate prediction: True
True target: 